In [28]:
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
load_dotenv()

DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "botgpt"
DB_PASSWORD = "pass"
DB_NAME = "bot_gpt_test"

In [35]:
def get_server_connection():
    return mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
    )

def get_connection():
    return mysql.connector.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
    )
    
def ensure_database():
    try:
        conn = get_server_connection()
        cursor = conn.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` CHARACTER SET utf8mb4;")
        conn.commit()
        cursor.close()
        conn.close()
        print(f"ensured {DB_NAME} database")
    except Error as e:
        print(f"Error ensuring database: {e}")
    finally:
        if conn and conn.is_connected():
            conn.close()
    
def init_db_test():
    ensure_database()

    try:
        conn = get_connection()
        cursor = conn.cursor()
    
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS sessions (
                id INT AUTO_INCREMENT PRIMARY KEY,
                session_id VARCHAR(255) NOT NULL UNIQUE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
        """)

        print("done")

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INT AUTO_INCREMENT PRIMARY KEY,
                session_id VARCHAR(255) NOT NULL,
                role VARCHAR(50) NOT NULL,
                content TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (session_id) REFERENCES sessions(session_id)
                    ON DELETE CASCADE
            );
        """)

        conn.commit()
        cursor.close()
        conn.close()
        print("Database tables created")
    except Error as e:
        print(f"Error creating tables: {e}")

In [36]:
init_db_test()

ensured bot_gpt_test database
done
Database tables created
